Preparing Data

<p> We will be combining data from three files </p>
<ol>
<li> We will select name, artist, genre, and number of grammy awards from grammySongs_1999-2019.csv. This information is not immediately available from the sheet, so we will group by artist and name and count the number of grammys each song has won. </li>
<li> We will select artist, name, peak position, average position, and weeks on chart from billboardHot100_1999-2019. </li>
<li> We will select artist, name, energy, danceability, tempo, and valence from data/songAtrributes_1999-2019.csv. </li>
</ol>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
# grammyAwards = grammyAwards[['Name', 'Artist', 'Genre', 'GrammyAward']]
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist']).count().reset_index()
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)
print(grammyAwards.shape)
grammyAwards.head(5)

In [ ]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
# billboardHot100.groupby(by=['Artist', 'Name']).min('Peak.position').max('Weeks.on.chart')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name']).agg({'Peak.position':'min', 'Weeks.on.chart':'max'})[['Peak.position','Weeks.on.chart']].reset_index()
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head(5)

In [ ]:
songAttributes = pd.read_csv('data/songAttributes_1999-2019.csv')
songAttributes = songAttributes[['Name', 'Artist', 'Energy', 'Danceability', 'Tempo', 'Valence']]
songAttributes = songAttributes.applymap(lambda s: s.upper() if type(s) == str else s)
songAttributes.head(5)

In [ ]:
songAttributes2 = pd.read_csv('data/songAttributesV2.csv')
# songAttributes.columns
songAttributes2 = songAttributes2[['song_title', 'artist', 'energy', 'danceability', 'tempo', 'valence']]
songAttributes2.rename(
    columns={"song_title":"Name", "artist":"Artist"}, inplace=True)
songAttributes2 = songAttributes2.applymap(lambda s: s.upper() if type(s) == str else s)
songAttributes2.head(5)

In [ ]:
# grammyAwards.set_index(['Name', 'Artist']).join(songAttributes.set_index(['Name', 'Artist']), how='left').reset_index()
# inProgress = grammyAwards.merge(songAttributes, on=['Name', 'Artist'], how='left').drop_duplicates(subset=['Name'], keep='first')
inProgress = grammyAwards.merge(songAttributes, on=['Name', 'Artist'], how='left').drop_duplicates(subset=['Name', 'Artist'], keep='first')
# inProgress
inProgress.merge(songAttributes2, on=['Name', 'Artist'], how='left')
# grammyAwards.drop_duplicates(subset=['Name'], keep='first')

In [ ]:
# a = np.array([])
# a = np.append(a, grammyAwards.Artist.unique())
grammyAwards.Artist.unique()
# a = np.append(a, billboardHot100.Artists.unique())
# np.savetxt('myfile.csv', a, delimiter=',', fmt="%s")
# len(a)
# grammyAwards.Artist.unique()